<!-- metadata: title -->
# Kenya Unit Trusts: Money Market Fund(KES) Analysis

<!-- metadata: subtitle -->
> ### Can Kenyan Money Market Funds gurantee capital preservation? 

<!-- metadata: date, type=date -->
**Published Date:**
2024-03-03

<!-- metadata: date-modified, type=date-->
**Date Modified:**
2024-05-05

<!-- metadata: keywords, type=array -->
**Keywords:**
  - money
  - kenya
  - unit-trusts
  - money-market-funds
  - MMF

<!-- metadata: categories, type=array -->
**Categories:**
  - kenya unit trusts
  - data science
  - money

## Description

<!-- metadata: description -->
Money market is a form of unit trust, where fund managers collect money from the group of investors, and invest on their behave. This reduces the overhead of managing your portfio and significantly reduces your risk. Let's statistically and critically analyze Money market funds in Kenya in general using publicly available information and hopefully paint a clearer picture of the state of unit trusts in Kenya. 

- What are the risk factors that exist? 
- What is the performance of money market funds?

## Abstract

<!-- metadata: abstract -->
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## Introduction

In Kenya, a good number of the population have a bank account, even if not the traditional bank account. It is estimated that by year 2029, 99.9% of Kenyans will be banked, ^[Population share with banking account in Kenya 2014-2029
Published by J. Degenhard, Jan 30, 2024. <https://www.statista.com/forecasts/1149636/bank-account-penetration-forecast-in-kenya>]. With increase in financial access, financial literacy and regular individuals will want to venture in the teritory of finantial assets huunting for higher interest rates. One of the most attractive entry level high-yield financial asset is the unit trust, specifically the money market funds. Its easy to start, deposit, withdraw, and its interest is daily compunding, while offering higher interest than any bank.

Unit trusts are required to publish their daily and effective annual yields on the daily newspapers, and thus an investor can therefore always confirm the value of their investment as well as benchmark the performance against other unit trusts. It is also mandatory for the scheme to issue monthly statements showing an investor’s investments position.^[Investing in Unit Trust Funds; 19 May, 2019; see <https://cytonn.com/topicals/investing-in-unit-1>]^[Publishing daily the price of units in at least two daily newspapers of national circulation,
in English. <https://cma.or.ke/wp-content/uploads/2023/03/CMA-Handbook-2021.pdf> ]

Unit Trust is required to have at least three trustees, ^[sshttps://cma.or.ke/wp-content/uploads/2023/03/CMA-Handbook-2021.pdf]

^[https://www.google.com/search?q=unit+trusts+are+mandated+to+inform+customers+of+yileds+CMA&sca_esv=d7f5b94d2e0e1e85&sca_upv=1&rlz=1C5GCEM_en&sxsrf=ADLYWIIy4tzw-iKWAAcD-6cN8riYTbVjdA%3A1720446046799&ei=XuyLZqa8MLSOxc8Pw42giA4&ved=0ahUKEwim4-X7yJeHAxU0R_EDHcMGCOEQ4dUDCA8&uact=5&oq=unit+trusts+are+mandated+to+inform+customers+of+yileds+CMA&gs_lp=Egxnd3Mtd2l6LXNlcnAiOnVuaXQgdHJ1c3RzIGFyZSBtYW5kYXRlZCB0byBpbmZvcm0gY3VzdG9tZXJzIG9mIHlpbGVkcyBDTUEyBxAhGKABGAoyBxAhGKABGApI1RFQvwFYtw5wAXgBkAEAmAHeAqABlQmqAQUyLTMuMbgBA8gBAPgBAZgCBaACownCAgoQABiwAxjWBBhHwgIFECEYnwWYAwDiAwUSATEgQIgGAZAGCJIHBzEuMC4zLjGgB5cW&sclient=gws-wiz-serpsssss]



## Data

Despite the requirements to have the daily yield published in two national newspapers, it is fairly tasking to find a good data source. Also, since there dont have to publish the newspapers that have digital alternatives, it also becomes difficult to capture all yileds without visiting the library and grabbing the actual physical copies, which makes this task very expesnsive. Again, getting historical data is also not free, most old newspaper records are sold, adding up the cost. Fortunately, since 2014, Cytonn has been doing free makket research, and publishing them at <https://cytonnreport.com/>. A few fund managers publish their daily yields at their websites, but without historical data; just the current day's yield, which effectively makes this data unuseful for analysis.

We settled on crawling and analysing the massive cytton research data that is publicly available since 2014. With over 600 reports with, crawl each of the reports in a way that doesnt break their systems, or deny others the service, extract the table, aggregate the table results and analyze the tables. We checked with Cytonn's terms of service. users are allowed to use their copyright data in accordance with fair use/dealing, ^[Reproduction is prohibited other than in accordance with the copyright notice, which forms part of these terms and conditions. <https://cytonn.com/terms-of-use> ]. To allow others to reproduce this analysys, we will save a copy of the crawled data for future researchers and data enthusisists.

## Analysis

### Imports

In [ ]:
import sys
import os

# Add parent directory to sys.path
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_dir)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pyppeteer.page import Page
import asyncio
import json
import io
from bs4 import BeautifulSoup, Tag
from urllib.request import urlopen
from pyppeteer.page import Request
from tqdm import tqdm
import re
import webbrowser
from typing import Callable
from copy import copy
from datetime import datetime, timedelta
from python_utils.web_screenshot import web_screenshot_async
from python_utils.get_browser import get_browser_page_async

### Fund Managers

Let's start of by listing all the certified fund mangers in Kenya by CMA.^[Approved Fund Managers by CMA. <https://www.cma.or.ke/licensees-market-players/>]

Lets start with a screenshot of the webpage.

In [ ]:
async def action(page: Page):
    await page.waitForSelector('ul.module-accordion')
    elements = await page.querySelectorAll('li .accordion-title')
    # Iterate through the elements to find the one containing 'FUND MANAGERS'
    for element in elements:
        text_content = await page.evaluate('(element) => element.textContent', element)
        if 'FUND MANAGERS' in text_content:
            # Click on the target element
            await element.click()
            break
    else:
        print('Element not found')
    await page.waitForSelector('li.current.builder-accordion-active')
    await asyncio.sleep(1)

await web_screenshot_async(
    "https://www.cma.or.ke/licensees-market-players/", 
    action = action,
    width=1500)

Let's query the "Fund Managers" table.

In [ ]:
url_response = urlopen("https://www.cma.or.ke/licensees-market-players/").read()
fund_managers_html_table = BeautifulSoup(url_response, "html.parser")\
    .find('span', string="FUND MANAGERS")\
        .parent\
            .parent\
                .parent\
                    .find('table')

fund_managers_df = pd.read_html(io.StringIO(str(fund_managers_html_table)))[0].dropna()
fund_managers_df

The address of `African Alliance Kenya Asset Management Limited` doesnt seem to be valid, lets populate it with `P.O. Box 27639 Nairobi 00506`

In [ ]:
fund_managers_df.loc[fund_managers_df['LICENCE NO.'] == 165, 'ADDRESS'] = 'P.O. Box 27639 Nairobi 00506'
fund_managers_df

### Add Columns

- Location Coordinates (and google map)
- Headquter location/address/country
- Launch Date
- Risk Profile
- Trustee
- Custodian
- Auditors
- Minimum Investment
- Minimum Additional Investment
- Initial Fee
- Annual Management Fee
- Distribution
- Asset Under Management/Market share
- Advertised Rate [Gross, Net]
- Duration to withdraw
- Security - joint account verification/validation
- has online portal
- Withdraw chargessssss
- Contacts
- where are the daily yileds published!

https://cytonnreport.com/research/cmmf-fact-sheet-june-2021

https://cytonnreport.com/research/cmmf-fact-sheet-may-2021

https://cytonnreport.com/research/cmmf-fact-sheet-april-2021

https://ke.cicinsurancegroup.com/mmf/

https://ke.britam.com/save-and-invest/business/invest/unit-trust-funds/unit-trusts

### Getting the Perormance

According to Capital Markets Authority, fund managers are required to publish their yields daily in a reputable newspaper. What this means accessing hostorical records requires a significant investment in time to collect, aggregate and validate the published yields. That not withstanding, two very important questions become obvious:
 - what is a reputbale newspaper? 
 - what yield do they publish, gross or net?
 - What picture does an anualized daily rate paint?
 - Is interest deffered or carried forward (reporting a convervative figure and retaining the rest for rainy day to preserve a picture of good performance)?

Luckily, one of the fund managers, one that has recently found its self in the courts far more often than it would have wished, does exelent investment and market research. It has a good-enough aggregate of fund managers and their performance weekly and monthly. We are going to crawl their data for analysis, and analyze it for manipulation.If the data comeout clean, we will analyze the trend of fund managers using the data.

### Cytton Research

We are going to crawl the data from cytonn research, https://cytonn.com/researches/categories/1

#### screen shots

Lests start with a view of weekly reports

In [ ]:
await web_screenshot_async(
    "https://cytonn.com/researches/categories/1",
    width=1000)

Here is the latest report

In [ ]:
await web_screenshot_async(
    "https://cytonnreport.com/research/cytonn-h12024-markets",
    width=1000)

Instead of directly crawling HTML from https://cytonn.com/researches/ page, we can instead crawl JSON from https://cytonnreport.com/research page, using the link https://cytonnreport.com/get/allreports.

In [ ]:
async def get_all_cytonn_reports(per_page_count: int = 10):
    page, browser = await get_browser_page_async()
    reports_url = "https://cytonnreport.com/get/allreports"
    reports_headers: dict = None
    reports_method: str = None
    async def catch_request(request: Request):
        nonlocal reports_headers
        nonlocal reports_method
        if request.url == reports_url:
            reports_headers = request.headers.copy()
            reports_method = request.method
            await request.continue_()
        else:
            await request.continue_()
    async def get_cytonn_reports(current_page: int):
        js_fetch_fn = f'''
            async () => {{
                const response = await fetch(
                    "{reports_url}", 
                    {{
                        "headers": {json.dumps(reports_headers)},
                        "referrer": "https://cytonnreport.com/research",
                        "referrerPolicy": "no-referrer-when-downgrade",
                        "body": {json.dumps(json.dumps(
                            {
                                "pagination": {
                                    "per_page": per_page_count, 
                                    "current_page": current_page
                                }
                            }))},
                        "method": "{reports_method}",
                        "mode": "cors",
                        "credentials": "include"
                    }});
                const json = await response.json();
                return json;
            }}
        '''
        response_json = await page.evaluate(js_fetch_fn)
        return response_json
    # Enable request interception
    await page.setRequestInterception(True)
    # Attach the request handler
    page.on('request', lambda request: asyncio.ensure_future(catch_request(request)))
    # Navigate to the desired URL
    await page.goto("https://cytonnreport.com/research")
    while not reports_headers:
        await asyncio.sleep(1)
    current_page = 1
    all_reports = []
    pbar: tqdm = None
    while True:
        reports_response = await get_cytonn_reports(current_page)
        reports = reports_response['data'] if reports_response else []
        if len(reports) > 0:
            total = reports_response['total']
            pbar = pbar or tqdm(total=total)
            pbar.update(len(reports))
            all_reports.extend(reports)
            last_page = reports_response['last_page']
            if last_page == current_page:
                break
            current_page += 1
        else:
            break
    await browser.close()
    if pbar:
        pbar.close()
    return all_reports

all_cytonn_reports = await get_all_cytonn_reports()
print(f'There are {len(all_cytonn_reports)} reports')

In [ ]:
# https://charanhu.medium.com/converting-pandas-dataframe-into-a-dataset-and-pushing-to-hugging-face-146e2ccac38d
all_cytonn_reports_df = pd.DataFrame(all_cytonn_reports)
# with pd.option_context(
#   'display.max_columns', None, 
#   'display.max_colwidth', 100):
#   display(all_cytonn_reports_df)
all_cytonn_reports_df

In [ ]:
all_cytonn_reports_df.iloc[0]

In [ ]:
all_cytonn_reports_df.columns

In [ ]:
def parse_and_expand_date(date_string: str):
    pattern = r"^(FY|Q[1-4]|H[1-2])'(\d{4})$"
    match = re.match(pattern, date_string, re.IGNORECASE)
    if not match:
        return None
    period, year = match.groups()
    year = int(year)
    if period.upper() == 'FY':
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)
    elif period.upper().startswith('Q'):
        quarter = int(period[1])
        start_month = (quarter - 1) * 3 + 1
        start_date = datetime(year, start_month, 1)
        end_date = start_date.replace(month=start_month + 2) + timedelta(days=32)
        end_date = end_date.replace(day=1) - timedelta(days=1)
    elif period.upper().startswith('H'):
        half = int(period[1])
        start_month = (half - 1) * 6 + 1
        start_date = datetime(year, start_month, 1)
        end_date = start_date.replace(month=start_month + 5) + timedelta(days=32)
        end_date = end_date.replace(day=1) - timedelta(days=1)
    return (start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

# Test the function
test_dates = ["FY'2019", "Q1'2020", "H1'2019", "fy'2018", "q3'2021", "h2'2022"]

for date in test_dates:
    result = parse_and_expand_date(date)
    if result:
        print(f"{date}: {result}")
    else:
        print(f"{date}: Invalid format")

In [ ]:
fund_manager_names = fund_managers_df.get('NAME').values.tolist()
class RecordInfo:
    TYPE_AUM: str = 'AUM' # 'AUM' - Assets Under Management
    TYPE_EAR: str = 'EAR' # 'EAR' - Effective Annual Rate
    def __init__(self, record_type: str, record_date: str, record_value: str, fund_manager: str):
        self.record_type = RecordInfo.__validate_record_type(record_type)
        self.record_date = parse_and_expand_date(record_date) or datetime.strptime(record_date, "%Y-%m-%d") or None
        self.record_value = RecordInfo.__validate_record_value(record_value)
        self.fund_manager = RecordInfo.__validate_fund_manager(fund_manager)
    def is_valid(self) -> bool:
        return bool(self.record_type and self.record_date and self.record_value and self.fund_manager)
    @staticmethod
    def __validate_fund_manager(value: str) -> str|None:
        first_name = lambda i: str(i).lower().strip().split(' ')[0]
        names = [i for i in fund_manager_names if first_name(i) in str(value).lower()]
        if len(names) == 1:
            return names[0]
        if 'total' not in str(value or '').lower():
            fund_manager_names.append(value)
            return value
        return None
    @staticmethod
    def __validate_record_value(value: str|float) -> str|None:
        if type(value) == float:
            return value
        # remove percentage sign
        value = value.rstrip('%')
        # remove comma and white space
        value = ''.join([i for i in value if i not in [' ', ',', '-']])
        return float(value) if value else None
    @staticmethod
    def __validate_record_type(value: str) -> str|None:
        value = value.upper()
        return value if value in [RecordInfo.TYPE_AUM, RecordInfo.TYPE_EAR] else None

table_columns_list: list[tuple[list[str], list[Callable[[pd.Series, dict], RecordInfo]]]] = [
    (
        ['Rank', 'Fund Manager', 'Effective Annual Rate'], 
        [
            # https://cytonnreport.com/research/kenyas-fy2024-2025-budget
            # https://cytonnreport.com/research/nairobi-metropolitan-area-serviced-apartments-report-2021
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['Effective Annual Rate'], row['Fund Manager'])
        ]
    ),
    (
        ['Rank', 'Fund Manager', 'Effective Annual'], 
        [
            # https://cytonnreport.com/research/cytonn-monthly-may-2024
            # https://cytonnreport.com/research/q12023-unit-trust-funds-performance-cytonn-monthly-july-2023
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['Effective Annual'], row['Fund Manager'])
        ]
    ),
    (
        ['Rank', 'Fund Manager', 'Daily Yield', 'Effective Annual Rate'], 
        [
            # Effective Annual Rate is better than Daily Yield: https://cytonnreport.com/research/cytonn-monthly-october-2021
            # https://cytonnreport.com/research/potential-effects-covid-19
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['Effective Annual Rate'], row['Fund Manager']),
        ]
    ),
    (
        ['no.', 'fund-managers', 'q1’2020-aum(kshs-mns)', 'q1’2020market-share', 'q2’2020-aum(kshs-mns)', 'q2’2020market-share', 'aum-growthq1’2020-–-q2’2020'], 
        [
            # https://cytonnreport.com/research/unit-trust-funds-performance-q2-2020
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, 'q1’2020', row['q1’2020-aum(kshs-mns)'], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, 'q2’2020', row['q2’2020-aum(kshs-mns)'], row['fund-managers']),
        ]
    ),
    (
        ['no.', 'fund-managers', "fy'2019-aum(kshs-mns)", 'q1’2020-aum(kshs-mns)', "aum-growth*fy'2019-–-q1’2020"], 
        [
            # https://cytonnreport.com/research/unit-trust-funds-perfomance-q1-2020-cytonn-weekly
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2019", row["fy'2019-aum(kshs-mns)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "q1’2020", row['q1’2020-aum(kshs-mns)'], row['fund-managers']),
        ]
    ),
    (
        ['no.', 'fund-managers', "fy'2018-aum-(kshs-mns)", "h1'2019-aum-(kshs-mns)", "aum-h1'2019-annualized-growth"], 
        [
            # https://cytonnreport.com/research/unit-trust-funds-performance
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2018", row["fy'2018-aum-(kshs-mns)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "h1'2019", row["h1'2019-aum-(kshs-mns)"], row['fund-managers']),
        ]
    ),
    (
        ['no.', 'money-market-fund', '2018-average-effective-annual-yield-p.a.'], 
        [
            # https://cytonnreport.com/research/investing-in-unit
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['2018-average-effective-annual-yield-p.a.'], row['money-market-fund'])
        ]
    ),
    (
        ['no.', 'fund-managers', 'q2’2020-aum', 'q2’2020', 'q3’2020-aum', 'q3’2020', 'aum-growth'], 
        [
            # https://cytonnreport.com/research/unit-trust-fund-performance-q3-1
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "Q2 2020", row['q2’2020-aum'], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "Q3 2020", row['q3’2020-aum'], row['fund-managers'])
        ]
    ),
    (
        ['rank', 'money-market-funds', 'effective-annual-rate-(average-q3’2020)'], 
        [
            # https://cytonnreport.com/research/unit-trust-fund-performance-q3-1
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['effective-annual-rate-(average-q3’2020)'], row['money-market-funds'])
        ]
    ),
    (
        ['no.', 'fund-managers', "fy'2018-aum(kshs-mns)", "fy'2019-aum(kshs-mns)", "aum-growthfy'2018---fy'2019"], 
        [
            # https://cytonnreport.com/research/fy2019-utf-performance
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2018", row["fy'2018-aum(kshs-mns)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2019", row["fy'2019-aum(kshs-mns)"], row['fund-managers']),
        ]
    ),
    (
        ['no.', 'fund-managers', "fy'2018-money-market-fund(kshs-mns)", "fy'2019-money-market-fund(kshs-mns)", "fy'2018-market-share", "fy'2019-market-share", 'variance'], 
        [
            # https://cytonnreport.com/research/fy2019-utf-performance
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2018", row["fy'2018-money-market-fund(kshs-mns)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2019", row["fy'2019-money-market-fund(kshs-mns)"], row['fund-managers']),
        ]
    ),
    (
        ['rank', 'money-market-funds', 'effective-annual-rate-(average-fy’2019)'], 
        [
            # https://cytonnreport.com/research/fy2019-utf-performance
            lambda row, record: RecordInfo(RecordInfo.TYPE_EAR, record['researchdate'], row['effective-annual-rate-(average-fy’2019)'], row['money-market-funds']),
        ]
    ),
    (
        ['no.', 'unit-trust-fund-manager', 'aum', '%-of-market-share'], 
        [
            # https://cytonnreport.com/research/investment-options-in-kenyan-market
            lambda row, record: RecordInfo(RecordInfo.TYPE_AUM, record['researchdate'], row['aum'], row['unit-trust-fund-manager']),
        ]
    ),
    (
        ['no.', 'fund-managers', "h1'2018-money-market-fund(kshs-mn)", 'fy’2018-money-market-fund-(kshs-mn)', "h1'2019-money-market-fund(kshs-mn)", "annualized-h1'2019-growth"], 
        [
            # https://cytonnreport.com/research/options-for-your-pension
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "h1'2018", row["h1'2018-money-market-fund(kshs-mn)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy’2018", row['fy’2018-money-market-fund-(kshs-mn)'], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "h1'2019", row["h1'2019-money-market-fund(kshs-mn)"], row['fund-managers']),
        ]
    ),
    (
        ['#', 'fund-managers', "h1'2018-money-market-fund-aum-(kshs-mn)", "fy'2018-money-market-fund-aum(kshs-mn)", "h1'2019-money-market-fund-aum(kshs-mn)", "annualized-h1'2019-aum-growth"], 
        [
            # https://cytonnreport.com/research/cytonn-monthly-august-2019
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "h1'2018", row["h1'2018-money-market-fund-aum-(kshs-mn)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "fy'2018", row["fy'2018-money-market-fund-aum(kshs-mn)"], row['fund-managers']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, "h1'2019", row["h1'2019-money-market-fund-aum(kshs-mn)"], row['fund-managers']),
        ]
    ),
    (
        [ 'No.', 'Collective Investment Schemes', "FY’2023 AUM", "FY’2023", "Q1'2024 AUM", "Q1’2024", 'AUM Growth'], 
        [
            # https://cytonnreport.com/research/q1-2024-unit-trust-funds-performance-note
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, 'FY’2023', row["FY’2023 AUM"], row['Collective Investment Schemes']),
            lambda row, _: RecordInfo(RecordInfo.TYPE_AUM, 'Q1’2024', row["Q1'2024 AUM"], row['Collective Investment Schemes'])
        ]
    ),
]

def column_name_match_fn(x: str, y:str) -> bool:
    x = x.strip().lower()
    y = y.strip().lower()
    z = lambda val: re.sub(r'\s+', ' ', val).replace(" ", "-")
    return x == y or z(x) == z(y)

def get_table(table: Tag):
    for tag in table.find_all(True):
        tag.attrs = {} # remove tags such as colspan and rowspan
    for (table_columns, extractor_callbacks) in table_columns_list:
        clean_up_tasks: list[Callable[[], None]] = []
        header_tr_s: list[Tag] = table.select('thead tr')
        is_match = False
        for header_tr in header_tr_s:
            header_td_s: list[Tag] = header_tr.find_all('td')
            is_match_new = \
                len(header_td_s) == len(table_columns)\
                and all(
                    [column_name_match_fn(header_td.get_text(strip=True), table_column) 
                     for header_td, table_column 
                     in zip(header_td_s, table_columns)])
            if not is_match_new:
                clean_up_tasks.append(header_tr.extract)
            is_match = is_match or is_match_new
        if is_match:
            try:
                [clean_up_task() for clean_up_task in clean_up_tasks]
                table_df = pd.read_html(io.StringIO(str(table)))[0]
                table_df.columns = table_columns
                return (table_df, extractor_callbacks)
            except Exception as e:
                print('error', e, table)
                continue
    return (None, None)

def is_valid_dataframe(df: pd.DataFrame | None) -> bool:
    return df is not None and not df.empty

def get_tables(html: str):
    parsed_html = BeautifulSoup(html, "html.parser")
    tables: list[Tag] = [table for table in parsed_html.find_all('table')]
    for table in tables:
        table_df, extractor_callbacks = get_table(copy(table))
        if is_valid_dataframe(table_df):
            yield (table_df, extractor_callbacks)

def extract_table_by_column_names(record: pd.Series):
    topics: list[dict] = record['topics']
    all_topic_bodies = ' '.join([topic.get('body') for topic in topics])
    raw_tables__extractor_callbacks = get_tables(all_topic_bodies)
    for raw_table, extractor_callbacks in raw_tables__extractor_callbacks:
        if len(extractor_callbacks) > 0:
            for callback in extractor_callbacks:
                extracted: list[RecordInfo] = [callback(raw_table_row, record) for _,raw_table_row in raw_table.iterrows()]
                yield raw_table, pd.DataFrame([vars(i) for i in extracted if i.is_valid()])
        else:
            yield raw_table, None

In [ ]:
# all_cytonn_reports_df.iloc[1]
# some pages have more than one table, ge: https://cytonnreport.com/research/unit-trust-fund-performance-q3-1
example_record = all_cytonn_reports_df.loc[
    all_cytonn_reports_df['url'] == 'https://cytonnreport.com/research/unit-trust-fund-performance-q3-1'
].iloc[0]
raw_and_extracted_dataframes = extract_table_by_column_names(example_record)

In [ ]:
raw_df, extracted_df = next(raw_and_extracted_dataframes)
extracted_df

In [ ]:
raw_df

#### Saving

In [ ]:
table_paths = 'extracted_tables'
os.makedirs(table_paths, exist_ok=True)
for _, record in tqdm(all_cytonn_reports_df.iterrows(), total=len(all_cytonn_reports_df)):
    raw_and_extracted_dataframes = extract_table_by_column_names(record)
    for _, extracted_df in raw_and_extracted_dataframes:
        if is_valid_dataframe(extracted_df):
            extracted_df.to_json(f'extracted_tables/{record.id}.json', orient='records')

In [ ]:
from glob import glob

dataframes = []
for filename in  glob(f'{table_paths}/*.json'):
    json_df = pd.read_json(filename)
    dataframes.append(json_df)
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df

In [ ]:
effective_annual_rate_df = combined_df[combined_df['record_type'] == 'EAR'].drop(columns=['record_type']).copy()
effective_annual_rate_df['record_date'] = pd.to_datetime(effective_annual_rate_df['record_date'])
pivot = effective_annual_rate_df.pivot(index='record_date', columns='fund_manager', values='record_value')
pivot

In [ ]:
pivot.plot(figsize=(20, 12))

<hr/>

In [ ]:
def row_predicate(row: pd.Series):
    row_str_value = ' '.join(str(column) for column in row).lower()
    unwanted_regexes = ['cic\s*group', 'cic\s*insurance', 'cic\s*academia']
    for unwanted_regex in unwanted_regexes:
        row_str_value = re.sub(unwanted_regex, "", row_str_value, flags=re.IGNORECASE)
    return 'cic' in row_str_value
def topics_tables_predicate(row: pd.Series):
    html = ' '.join([topic.get('body') for topic in row['topics']])
    parsed_html = BeautifulSoup(html, "html.parser")
    tables: list[Tag] = [table for table in parsed_html.find_all('table')]
    tables_str_value = ' '.join(str(table) for table in tables).lower()
    unwanted_regexes = ['cic\s*group', 'cic\s*insurance', 'cic\s*academia']
    for unwanted_regex in unwanted_regexes:
        tables_str_value = re.sub(unwanted_regex, "", tables_str_value, flags=re.IGNORECASE)
    return 'cic' in tables_str_value

matched_records = []
for index, record in tqdm(all_cytonn_reports_df.iterrows(), total=len(all_cytonn_reports_df)):
    raw_and_extracted_dataframes = extract_table_by_column_names(record)
    extracts = list(raw_and_extracted_dataframes)
    is_row_match = row_predicate(record)
    is_topics_match = topics_tables_predicate(record)
    matched_records.append((index, len(extracts), is_row_match, is_topics_match))

In [ ]:
indexes_with_cic = [index for index, tables, is_row_match, is_topics_match in matched_records if tables == 0 and is_row_match and is_topics_match]
indexes_with_cic

<hr/>

In [ ]:
len(indexes_with_cic)

In [ ]:
webbrowser.get("/usr/bin/google-chrome %s")
for index in indexes_with_cic[20:]:
    url = str(all_cytonn_reports_df.loc[index, 'url'])
    webbrowser.open(url)

In [ ]:
row = all_cytonn_reports_df.loc[
    all_cytonn_reports_df['url'] == 'https://cytonnreport.com/research/unit-trust-fund-performance-q3-1'
].iloc[0]
dfs = extract_table_by_column_names(row)

In [ ]:
dfs[0]

<hr/>

In [ ]:
row = all_cytonn_reports_df.loc[
    all_cytonn_reports_df['url'] == 'https://cytonnreport.com/research/q1-2024-unit-trust-funds-performance-note'
].iloc[0]
table_columns_list = [
    (
        [ 'No.', 'Collective Investment Schemes', "FY’2023 AUM", "FY’2023", "Q1'2024 AUM", "Q1’2024", 'AUM Growth'], 
        [
            # https://cytonnreport.com/research/q1-2024-unit-trust-funds-performance-note
            lambda row, _: RecordInfo("AUM", 'FY’2023', row["FY’2023 AUM"], row['Collective Investment Schemes']),
            lambda row, _: RecordInfo("AUM", 'Q1’2024', row["Q1'2024 AUM"], row['Collective Investment Schemes'])
        ]
    ),
]
dfs = extract_table_by_column_names(row)

In [ ]:
one = next(dfs)

In [ ]:
one[1]